In [9]:
import numpy as np
import pandas as pd
from pathlib import Path
import holidays
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [10]:
df_train = pd.read_parquet(Path('data') / 'train.parquet')
df_test = pd.read_parquet(Path('data') / 'final_test.parquet')

X_train = df_train.drop(columns=['log_bike_count', 'bike_count'])
y_train = df_train['log_bike_count']
X_test = df_test
X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496827 entries, 48321 to 929187
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 496827 non-null  category      
 1   counter_name               496827 non-null  category      
 2   site_id                    496827 non-null  int64         
 3   site_name                  496827 non-null  category      
 4   date                       496827 non-null  datetime64[ns]
 5   counter_installation_date  496827 non-null  datetime64[ns]
 6   coordinates                496827 non-null  category      
 7   counter_technical_id       496827 non-null  category      
 8   latitude                   496827 non-null  float64       
 9   longitude                  496827 non-null  float64       
dtypes: category(5), datetime64[ns](2), float64(2), int64(1)
memory usage: 25.1 MB
<class 'pandas.core.frame.DataFram

In [11]:
def _encode_dates(X):
    X = X.copy()
    X['date'] = pd.to_datetime(X['date'])
    X['month'] = X['date'].dt.month
    X['day'] = X['date'].dt.day
    X['weekday'] = X['date'].dt.weekday
    X['hour'] = X['date'].dt.hour

    fr_holidays = holidays.France(years=[2020, 2021])
    def is_holiday(date):
        weekday = date.weekday()
        if weekday > 4 or date in fr_holidays:
            return 1
        else:
            return 0
    X['is_holiday'] = X['date'].apply(is_holiday)
    return X


def _merge_external_data(X):
    file_path = Path('data') / 'external_data.csv'
    df_ext = pd.read_csv(file_path, parse_dates=['date'])
    X = X.copy()
    X['date'] = X['date'].astype('datetime64[ns]')
    X['orig_index'] = np.arange(X.shape[0])
    cols_to_merge = ['date', 'pmer', 'tend', 'cod_tend', 'tend24',
                                        'dd', 'ff', 't', 'td', 'u', 'vv',  
                                        'n', 'pres', 'raf10', 'ww', 'nbas',
                                        'ht_neige', 'rr1', 'rr6',]
    X = pd.merge_asof(
        X.sort_values('date'), df_ext[cols_to_merge].sort_values('date'), on='date'
    )  
    X = X.sort_values('orig_index')
    del X['orig_index']
    for col in cols_to_merge:
        if X[col].isnull().any():
            X[col].fillna(X[col].mean(), inplace=True)
    return X


def one_hot_encode_and_concat(X):
    categorical_columns = ['counter_id']
    one_hot = OneHotEncoder(handle_unknown='ignore')
    one_hot_encoded_data = one_hot.fit_transform(X[categorical_columns])
    one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data.toarray(), 
            columns=one_hot.get_feature_names_out(categorical_columns))
    X_dropped = X.drop(columns=categorical_columns)
    X_encoded = pd.concat([X_dropped.reset_index(drop=True), one_hot_encoded_df.reset_index(drop=True)], axis=1)
    del X_encoded['date']

    return X_encoded


def _drop_columns(X):
    res = X.copy()
    res = res.drop(columns=['counter_name',
        'coordinates',
        'site_name',
        'site_id',
        'counter_technical_id',
        'counter_installation_date',
        'latitude', 
        'longitude'
        ])
    return res

In [12]:
def preprocessing(X):
    X = _encode_dates(X)
    X = _merge_external_data(X)
    X = one_hot_encode_and_concat(X)
    X = _drop_columns(X)
    return X

X_train = preprocessing(X_train)
X_test = preprocessing(X_test)

In [16]:
regressor = XGBRegressor(
    learning_rate=0.15,
    max_depth=10,
    n_estimators=300
)

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', regressor)
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.15,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [17]:
prediction = pipeline.predict(X_test)
results = pd.DataFrame(
    dict(
        Id=np.arange(prediction.shape[0]),
        log_bike_count=prediction,
    )
)

results.to_csv("submission.csv", index=False)
print("CSV file created: submission.csv")

CSV file created: submission.csv
